# Учебный проект «Парсинг сайта» 20250430

#### pip install requests beautifulsoup4 lxml

## Импорт необходимых библиотек import requests # Для отправки HTTP-

In [1]:
import requests # Для отправки HTTP-запросов к сайту
from bs4 import BeautifulSoup # Для парсинга HTML-страниц
import pprint # Для красивого вывода данных в консоль
from time import sleep # Для добавления задержек между запросами
import pandas as pd # Для работы с данными в табличном формате
import os # Для работы с файловой системой
from openpyxl import Workbook, load_workbook # Для работы с Excel-файлами

#### Базовый домен сайта (используется для формирования полных URL)

In [2]:
domen = 'https://scrapingclub.com'

#### Стартовая URL-страница для парсинга

In [3]:
url = 'https://scrapingclub.com/exercise/list_basic/?page=1'

#### Заголовки HTTP-запроса (имитируем браузер)

In [4]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

#### Путь к файлу Excel для сохранения результатов для построчного последовательного заполнения спарсенными карточками товаров

In [5]:
output_file = r'D:\PROJECTS\PROJECT_20220926_SKILL_FACTORY\IDE\WILDBERRIES\data_XlsxWriter_line.xlsx'

#### Проверка существования файла и его создание/загрузка: ... если файл есть — загружаем, если нет — создаём

In [6]:
if not os.path.exists(output_file):
    # Если файл не существует - создаем новый
    wb = Workbook()  # Создаем новую книгу Excel
    ws = wb.active  # Получаем активный лист
    ws.title = "Sheet1"  # Переименовываем лист
    # Добавляем заголовки столбцов
    ws.append(['name', 'price', 'text', 'url_img']) # Заголовки
    wb.save(output_file)  # Сохраняем файл
    print(f"{os.path.basename(output_file)} создан")
else:
    # Если файл существует - загружаем его
    wb = load_workbook(output_file)
    ws = wb.active
    print(f"{os.path.basename(output_file)} создан повторно")

data_XlsxWriter_line.xlsx создан повторно


## Отправляем GET-запрос к стартовой странице

In [7]:
response = requests.get(url, headers = headers)

#### Создаем объект BeautifulSoup для парсинга HTML - здесь использую парсер lxml, но в некоторых (для малых объёмов данных) случаях можно html.parser вместо lxml

In [8]:
soup = BeautifulSoup(response.text, 'lxml')

## Находим блок пагинации

In [9]:
pagination = soup.find('nav', class_='pagination')

#### Получаем все элементы пагинации ( кoд страницы сайта и смотрим теги "a" в углоавых скобках) pagination.find_all('a') возвращает список ссылок на страницы: 1 2 3 ... Next.

In [10]:
pages = pagination.find_all('a')

#### Последний элемент перед "Next" — это номер последней страницы или иначе предпоследний элемент (pages[-2]) содержит номер последней страницы и потом преобразуем его .text в int.

In [11]:
last_page_number = int(pages[-2].text)
print("\nКоличество страниц:", last_page_number,'\n')


Количество страниц: 6 



In [12]:
def get_url():
    """Генератор, который возвращает URL-адреса карточек товаров со всех страниц"""
    """Функция с yield, которая возвращает URL по одному - это позволяет ..."""
    """... обрабатывать страницы последовательно без загрузки всех URL в память"""

    for count in range(1, last_page_number):

        # Формируем URL для каждой страницы
        url = f'https://scrapingclub.com/exercise/list_basic/?page={count}'

        # Отправляем запрос к странице
        response = requests.get(url, headers = headers)

        # Парсим HTML страницы
        soup = BeautifulSoup(response.text, 'lxml') # в некоторых случаях можно html.parser вместо lxml

        data = soup.find_all('div')

        # Находим все карточки товаров на странице
        data = soup.find_all('div', class_ = 'w-full rounded border')

        # Для каждой карточки извлекаем информацию
        for i in data:

            # Извлекаем название товара (удаляем лишние пробелы и переносы строк)
            name = i.find('h4').text.replace(' ', '').replace('\n', '')

            # Извлекаем цену товара
            price = i.find('h5').text

            # Формируем полный URL изображения (добавляем домен)
            url_img = domen + i.find('img', class_ = 'card-img-top img-fluid').get('src')

            # Формируем полный URL страницы товара
            card_url = domen + i.find('a').get('href')
            
            # Возвращаем URL страницы товара (генератор)
            yield card_url

## Список (словарь) для хранения данных обо всех товарах по каждой карточке товара

In [13]:
data_list = []

#### Итерируем по всем URL карточек товаров

In [14]:
for card_url in get_url():

    # Отправляем запрос к странице товара
    response = requests.get(card_url, headers=headers)

    # Парсим HTML страницы товара
    soup = BeautifulSoup(response.text, 'lxml')

    # Находим основной блок с информацией о товаре
    data = soup.find('div', class_='my-8 w-full rounded border')
    
    if data: # Если блок найден
        try:
            # Извлекаем название товара
            name = data.find('h3', class_='card-title').text.strip()
            # Извлекаем цену товара
            price = data.find('h4', class_='my-4 card-price').text.strip()
            # Извлекаем описание товара
            text = data.find('p', class_='card-description').text.strip()
            # Формируем полный URL изображения
            url_img = domen + data.find('img').get('src')

            # Добавляем данные о товаре в список - пополнение словаря
            data_list.append({
                'name': name,
                'price': price,
                'text': text,
                'url_img': url_img
            })

            # Добавляем данные в Excel-файл (построчно)
            ws.append([name, price, text, url_img])
            # Сохраняем изменения в файле
            wb.save(output_file)

        except AttributeError as e:
            # Если какой-то элемент не найден
            print(f"Некоторые данные не найдены на странице: {card_url}")
            print(f"Ошибка: {e}")
    else:
        print(f"Блок данных не найден на странице: {card_url}")

#### Создаем DataFrame из собранных данных

In [15]:
df = pd.DataFrame(data_list, columns=['name', 'price', 'text', 'url_img'])

#### Выводим DataFrame в консоль

In [16]:
print(df)

                           name   price  \
0                   Short Dress  $24.99   
1              Patterned Slacks  $29.99   
2           Short Chiffon Dress  $49.99   
3        Off-the-shoulder Dress  $59.99   
4                    V-neck Top  $24.99   
5           Short Chiffon Dress  $49.99   
6                    V-neck Top  $24.99   
7                    V-neck Top  $24.99   
8              Short Lace Dress  $59.99   
9                  Fitted Dress  $34.99   
10              V-neck Jumpsuit  $69.99   
11                Chiffon Dress  $54.99   
12      Skinny High Waist Jeans  $39.99   
13      Super Skinny High Jeans  $19.99   
14       Oversized Denim Jacket  $19.99   
15             Short Sweatshirt  $24.99   
16      Long-sleeved Jersey Top  $12.99   
17      Skinny High Waist Jeans  $39.99   
18             Short Sweatshirt  $24.99   
19      Long-sleeved Jersey Top  $12.99   
20      Long-sleeved Jersey Top  $12.99   
21                 Jersey Dress  $19.99   
22         

#### Сохраняем DataFrame в отдельный Excel-файл

In [17]:
df.to_excel(r'D:\PROJECTS\PROJECT_20220926_SKILL_FACTORY\IDE\WILDBERRIES\data_XlsxWriter.xlsx', index=False)

#### Завершаем работу программы

In [18]:
quit()